# Setup for invoking Anthropic and Titan Family of models via SAP GenAI Hub on SAP AI core

## Part 1: Use AI Core REST APIs

Use this notebook to invoke the AI Core REST APIs to send your payloads into LLMs hosted on SAP GenAI hub. The documentation for the APIs are provided [here](https://api.sap.com/api/AI_CORE_API/resource/Deployment).


## Part 2: Use SAP Generative AI Hub SDK
Sample code to invoke Anthropic and Titan Family of models using the ChatBedrock interface in the SAP Generative AI Hub SDK. Documentation [here](https://pypi.org/project/generative-ai-hub-sdk/). 



## Part 1:
### Step 1: Load your AI Core credentials

After following the steps in the previous notebook to setup the AI Core instance, Configurations and Deployments for Bedrock models, you can invoke the models via the following approach. 

First, you should have your AI core creadentials in your ~/.aicore/config.json. If you do not, get it by creating a service key from [here](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/create-service-key). We will read and consume the data from this location into temporary environment files for this notebook.

A sample config.json is below:

```sh
$ cat ~/.aicore/config.json  
{
  "AICORE_AUTH_URL": "<>.authentication.us10.hana.ondemand.com",
  "AICORE_CLIENT_ID": "sb-b...64",
  "AICORE_CLIENT_SECRET": "21...xc=",
  "AICORE_RESOURCE_GROUP": "default",
  "AICORE_BASE_URL": "https://api.ai.prod.<>.ondemand.com"
}
```

In [9]:
import os
import json
import base64

def load_config():
    with open(os.path.expanduser('~/.aicore/config.json')) as f:
        config = json.load(f)
    for key, value in config.items():
        os.environ[key] = value

Instructions: Uncomment the line which has the model you would like to use.


In [11]:
# from https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/models-and-scenarios-in-generative-ai-hub or uncomment from below

# Multimodal models:
os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3.5-sonnet"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-opus"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-sonnet"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-haiku"

#Text based models:
# os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-text-lite"
# os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-text-express"

# Embedding models:
# os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-embed-text"


Optional Step: If you know the deployment ID from AI Launchpad already, you can uncomment below. This way you don't need to speicify the model name in the previous step.

In [27]:
# OPTIONAL:
# Uncomment the below line after pasting your deployment ID if you already know it
# os.environ["DEPLOYMENT_ID"] = "<>"

### Step 2: Create Utility functions
Creating a utility function to send API calls to AI Core instance repeatedly. You will need to install requests library in the terminal 
```sh
pip install requests
```

In [2]:
import requests
import sys
import logging
import time
from util.logging import initLogger

TIME_RETRY_API_CALL = 20
TIMEOUT_API_CALL = 3600

log = logging.getLogger(__name__)
initLogger()


# Function to call a rest API
def call_api(
    type: str, url: str, headers: dict, data: dict = None, message: str = None
):
    timeNeeded = 0
    while timeNeeded < TIMEOUT_API_CALL:
        try:
            r = None
            # Send the request to retrieve the access token
            if type == "POST":
                r = requests.post(url=url, headers=headers, data=data)
            elif type == "GET":
                r = requests.get(url=url, headers=headers)
            # if the response is OK, return the JSON response
            if r.ok is True:
                log.success(f"{message}")
                return r.json()
            else:
                log.info(
                    f"response ({message}): {r.status_code} ({r.reason}): {r.text}"
                )
                log.warning(
                    f"Could not {message}! Re-trying in {TIME_RETRY_API_CALL} seconds..."
                )
                time.sleep(TIME_RETRY_API_CALL)
                timeNeeded += TIME_RETRY_API_CALL
        except requests.exceptions.RequestException as e:
            log.warning(str(e))
            log.error(f"Could not {message}! Exiting...")
            sys.exit(1)
    log.error(f"Could not {message} after {TIMEOUT_API_CALL} seconds! Exiting...")


Creating a template to hold the deploymentURL and other imporatant parameters


In [3]:
from dataclasses import dataclass
from json import JSONEncoder


class AiCoreMetadataJsonEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__


@dataclass
class AiCoreMetadataDefinition:
    authUrl: str
    clientId: str
    clientSecret: str
    apiBase: str
    resourceGroup: str
    targetAiCoreModel: str
    apiAccessToken: str
    deploymentUrl : str

    def __getitem__(self, item):
        return getattr(self, item)

Creating a dataClass that will initiate with the environment variables that have the AI Core credentials as well as the target model

In [4]:
@dataclass
class AiCoreMetadata(AiCoreMetadataDefinition):
    def __init__(self):

        load_config()

        self.authUrl = os.environ.get("AICORE_AUTH_URL")
        self.clientId = os.environ.get("AICORE_CLIENT_ID")
        self.clientSecret = os.environ.get("AICORE_CLIENT_SECRET")
        self.resourceGroup = os.environ.get("AICORE_RESOURCE_GROUP")
        self.apiBase = os.environ.get("AICORE_BASE_URL")
        self.targetAiCoreModel = os.environ.get("TARGET_AI_CORE_MODEL")
        self.apiAccessToken = get_api_access_token(self)
        self.deploymentUrl = get_deployment_details_for_model(self)

Creating a utility function that will get the API Access Token

In [5]:
def get_api_access_token(aiCoreMetadata: AiCoreMetadataDefinition) -> str:
    clientId = aiCoreMetadata.clientId
    clientSecret = aiCoreMetadata.clientSecret
    authUrl = aiCoreMetadata.authUrl

    # Create the authorization string
    authorizationString = f"{clientId}:{clientSecret}"
    # Encode the authorization string
    byte_data = authorizationString.encode("utf-8")
    # Base64 encode the byte data
    clientSecretBase64 = base64.b64encode(byte_data).decode("utf-8")

    # Create the URL to retrieve the access token
    aiCoreLocation = f"{authUrl}/oauth/token?grant_type=client_credentials"
    # Create the headers for the request
    headers = {"Authorization": f"Basic {clientSecretBase64}"}

    response = call_api(
        "POST",
        aiCoreLocation,
        headers,
        None,
        "retrieve access token from AI Core system",
    )
    # json_response = json.dumps(response, indent=2)

    return response["access_token"]

Once we have the API Access token, lets get the deployment URL that corresponds to the Model you want to use.


In [28]:
# Retrieve the deployment URL from the AI Core system metadata
def get_deployment_details_for_model(aiCoreMetadata: AiCoreMetadataDefinition):
    apiBase = aiCoreMetadata.apiBase
    token = aiCoreMetadata.apiAccessToken
    resourceGroup = aiCoreMetadata.resourceGroup

    # Create the URL to get the deployment 
    aiCoreLocation = f"{apiBase}/v2/lm/deployments"
    # Create the headers for the request
    headers = {}
    headers["AI-Resource-Group"] = resourceGroup
    headers["Authorization"] = f"Bearer {token}"
    allDeploymentDetails = None

    timeNeeded = 0
    message = f"retrieve deployment details for model id {aiCoreMetadata.targetAiCoreModel}"
    while timeNeeded < TIMEOUT_API_CALL:
        # Send the request to get the list of deployments
        response = call_api("GET", aiCoreLocation, headers, None, message)
        json_response = json.dumps(response, indent=2)
        # log.check(
        #         f"API response from retrieveing deployment details for model id {aiCoreMetadata.targetAiCoreModel}:\n{json_response}"
        # )

        allDeploymentDetails = response
    
        for resource in allDeploymentDetails["resources"]:
            model_name = resource["details"]["resources"]["backend_details"]["model"]["name"]
            if model_name == aiCoreMetadata.targetAiCoreModel:
                return resource["deploymentUrl"]
        
        log.warning(
                f"Could not {message} Re-trying in {TIME_RETRY_API_CALL} seconds..."
            )

        time.sleep(TIME_RETRY_API_CALL)
        timeNeeded += TIME_RETRY_API_CALL

    log.error(
        f"Could not retrieve deployment details for id '{aiCoreMetadata.targetAiCoreModel}'! Exiting..."
    )
    sys.exit(1)

Another utility function to get deployment details in case deployment ID is provided. 

In [29]:
# Retrieve the deployment URL from the AI Core system metadata
def get_deployment_details(aiCoreMetadata: AiCoreMetadataDefinition, deploymenId: str):
    apiBase = aiCoreMetadata.apiBase
    token = aiCoreMetadata.apiAccessToken
    resourceGroup = aiCoreMetadata.resourceGroup

    # Create the URL to create the configuration
    aiCoreLocation = f"{apiBase}/v2/lm/deployments/{deploymenId}"
    # Create the headers for the request
    headers = {}
    headers["AI-Resource-Group"] = resourceGroup
    headers["Authorization"] = f"Bearer {token}"
    deploymentDetails = None

    timeNeeded = 0
    message = f"retrieve deployment details for deployment id {deploymenId}"
    while timeNeeded < TIMEOUT_API_CALL:
        # Send the request to create the deployment
        response = call_api("GET", aiCoreLocation, headers, None, message)
        # json_response = json.dumps(response, indent=2)

        deploymentDetails = response
        deploymentUrl = deploymentDetails["deploymentUrl"]
        if deploymentUrl != "":
            log.success(f"AI Core deployment id '{deploymenId}' is now accessible!")
            return deploymentDetails
        else:
            log.warning(
                f"Could not {message} (deployment not finished)! Re-trying in {TIME_RETRY_API_CALL} seconds..."
            )

            time.sleep(TIME_RETRY_API_CALL)
            timeNeeded += TIME_RETRY_API_CALL

    log.error(
        f"Could not retrieve deployment details for id '{deploymenId}'! Exiting..."
    )
    sys.exit(1)

Let's instantiate our class. This step will populate the internal environment variables, execute the access token get utility function and also get the deployment URL for the model specified.

Optional Step - execute only if you directly want to specify the deployment URL 

In [30]:
# In case you have provided deployment ID directly instead of model id 
if ai_core_metadata.deploymentUrl == None:
    deploymentDetails = get_deployment_details(ai_core_metadata, os.environ["DEPLOYMENT_ID"])
    ai_core_metadata.deploymentUrl = deploymentDetails["deploymentUrl"]

This is the utility function to send your input prompt to GenAI Hub

In [31]:
# Retrieve the available AI models from the AI Core system
def invoke(aiCoreMetadata: AiCoreMetadataDefinition, 
           data) -> str:

    token = aiCoreMetadata.apiAccessToken
    deploymentUrl = aiCoreMetadata.deploymentUrl
    
    # Create the URL to retrieve the available AI models
    aiCoreLocation = f"{deploymentUrl}/invoke"
    # Create the headers for the request
    headers = {
        "AI-Resource-Group": aiCoreMetadata.resourceGroup,
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}",
    }
    
    response = call_api(
        "POST",
        aiCoreLocation,
        headers,
        json.dumps(data),
        "sending invoke",
    )
    json_response = json.dumps(response, indent=2)

    return response

## Step 3 Send the prompt to the GenAI Hub

Lets define a payload in the messages format

In [32]:
# For Anthropic Claude models

# Multimodal models:
os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3.5-sonnet"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-opus"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-sonnet"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-haiku"

data = {}
messages = [
        {
            "role": "user",
            "content": "Hello, What is the capital of United States?"
        }
    ]
data["anthropic_version"] = "bedrock-2023-05-31"
data["max_tokens"] = 1000
data["messages"] = messages


In [33]:
# Extract the metadata for the AI Core system
ai_core_metadata = AiCoreMetadata()

[2024-10-02 16:22:17] SUCCESS: retrieve access token from AI Core system
[2024-10-02 16:22:18] SUCCESS: retrieve deployment details for model id anthropic--claude-3.5-sonnet


In [34]:
invoke(aiCoreMetadata= ai_core_metadata, 
       data=data)

[2024-10-02 16:22:24] SUCCESS: sending invoke


{'content': [{'text': "The capital of the United States is Washington, D.C. (District of Columbia). It's located on the east coast of the country, between the states of Maryland and Virginia. Washington, D.C. is the seat of the federal government and home to many important national monuments, museums, and government buildings, including:\n\n1. The White House (residence and office of the President)\n2. The United States Capitol (meeting place of Congress)\n3. The Supreme Court\n4. Various federal departments and agencies\n\nIt's worth noting that Washington, D.C. is not a state, but a federal district created specifically to serve as the national capital. It was founded in 1790 and named after the first U.S. President, George Washington.",
   'type': 'text'}],
 'id': 'msg_bdrk_01UNu67BYNsAzM4tFS7Qw8Jx',
 'model': 'claude-3-5-sonnet-20240620',
 'role': 'assistant',
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'type': 'message',
 'usage': {'input_tokens': 17, 'output_tokens': 162

In [35]:
# For Amazon Titan Text Models

#Text based models:
os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-text-lite"
# os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-text-express"

data = {}

data = {
    "inputText": "What is the capital of United States?",
    "textGenerationConfig": {
        "maxTokenCount": 1000,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1
     }
}

In [36]:
# Extract the metadata for the AI Core system
ai_core_metadata = AiCoreMetadata()

[2024-10-02 16:22:28] SUCCESS: retrieve access token from AI Core system
[2024-10-02 16:22:28] SUCCESS: retrieve deployment details for model id amazon--titan-text-lite


In [37]:
invoke(aiCoreMetadata= ai_core_metadata, 
       data=data)

[2024-10-02 16:22:30] SUCCESS: sending invoke


{'inputTextTokenCount': 8,
 'results': [{'completionReason': 'FINISH',
   'outputText': '\nWashington, D.C. is the capital of the United States.',
   'tokenCount': 16}]}

In [38]:
# For Amazon Titan Embedding Models
os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-embed-text"

data = {}

data = {
    "inputText": "What is the capital of United States?"
}

In [39]:
# Extract the metadata for the AI Core system
ai_core_metadata = AiCoreMetadata()

[2024-10-02 16:22:33] SUCCESS: retrieve access token from AI Core system
[2024-10-02 16:22:33] SUCCESS: retrieve deployment details for model id amazon--titan-embed-text


Finally get a result

In [ ]:
invoke(aiCoreMetadata= ai_core_metadata, 
       data=data)


## Part 2: Use SAP Generative AI Hub SDK
Sample code to invoke Anthropic and Titan Family of models using the ChatBedrock interface in the SAP Generative AI Hub SDK. Documentation [here](https://pypi.org/project/generative-ai-hub-sdk/). 

In [ ]:
!pip -q install generative-ai-hub-sdk langchain_aws

In [2]:
from gen_ai_hub.proxy.langchain.amazon import ChatBedrock
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

In [55]:
proxy_client = get_proxy_client("gen-ai-hub")
llm_via_bedrock_on_GenAIHub = ChatBedrock(
        model_name="anthropic--claude-3.5-sonnet",
        proxy_client=proxy_client,
        temperature=0,
    )

In [56]:
messages = [
        ( "system", "You are a helpful assistant" ),
        ( "user", "Hello, What are the top visitor destinations in NYC?" )
        ]

response_body = llm_via_bedrock_on_GenAIHub.invoke(messages)
print(response_body.content)

New York City offers numerous popular attractions for visitors. Some of the top destinations include:

1. Statue of Liberty and Ellis Island
2. Empire State Building
3. Central Park
4. Times Square
5. Metropolitan Museum of Art (The Met)
6. Broadway Theater District
7. Rockefeller Center and Top of the Rock Observation Deck
8. 9/11 Memorial and Museum
9. High Line
10. One World Trade Center and One World Observatory
11. Brooklyn Bridge
12. Grand Central Terminal
13. Museum of Modern Art (MoMA)
14. Fifth Avenue (for shopping)
15. American Museum of Natural History
16. St. Patrick's Cathedral
17. Coney Island
18. Chinatown and Little Italy
19. The Guggenheim Museum
20. Hudson Yards and The Vessel

These attractions offer a mix of history, culture, art, architecture, and entertainment, making New York City a diverse and exciting destination for visitors.


In [53]:
llm_via_bedrock_on_GenAIHub_2 = ChatBedrock(
        model_name="amazon--titan-text-lite",
        proxy_client=proxy_client,
        temperature=0,
    )

In [54]:
messages = [
        ( "system", "You are a helpful assistant" ),
        ( "user", "Hello, What is the capital of United States?" )
        ]

response_body = llm_via_bedrock_on_GenAIHub_2.invoke(messages)
print(response_body.content)

 Hello, The capital of the United States is Washington, D.C.
